# Building the Streaming Dataflow Accelerator

<font color="red">**Live FINN tutorial:** We recommend clicking **Cell -> Run All** when you start reading this notebook for "latency hiding".</font>

**Important: This notebook depends on the 1-train-mlp-with-brevitas notebook because we are using models that were created by that notebook. So please make sure the needed .onnx files are generated prior to running this notebook.**

<img align="left" src="finn-example.png" alt="drawing" style="margin-right: 20px" width="250"/>

In this notebook, we'll use the FINN compiler generate an FPGA accelerator with a streaming dataflow architecture from our quantized MLP for the cybersecurity task. The key idea in such architectures is to parallelize across layers as well as within layers by dedicating a proportionate amount of compute resources to each layer, illustrated on the figure to the left. You can read more about the general concept in the [FINN](https://arxiv.org/pdf/1612.07119) and [FINN-R](https://dl.acm.org/doi/pdf/10.1145/3242897) papers. This is done by mapping each layer to a Vivado HLS description, parallelizing each layer's implementation to the appropriate degree and using on-chip FIFOs to link up the layers to create the full accelerator.

These implementations offer a good balance of performance and flexibility, but building them by hand is difficult and time-consuming. This is where the FINN compiler comes in: it can build streaming dataflow accelerators from an ONNX description to match the desired throughput.

## Outline
-------------

1. [Introduction to  `build_dataflow` Tool](#intro_build_dataflow) 
2. [Understanding the Build Configuration: `DataflowBuildConfig`](#underst_build_conf)     
    2.1.[Output Products](#output_prod)   
    2.2.[Configuring the Board and FPGA Part](#config_fpga)   
    2.3 [Configuring the Performance](#config_perf)    
4. [Launch a Build: Only Estimate Reports](#build_estimate_report)
5. [Launch a Build: Stitched IP, out-of-context synth and rtlsim Performance](#build_ip_synth_rtlsim)
6. [(Optional) Launch a Build: PYNQ Bitfile and Driver](#build_bitfile_driver)
7. [(Optional) Run on PYNQ board](#run_on_pynq)

## Introduction to  `build_dataflow` Tool <a id="intro_build_dataflow"></a>

Since version 0.5b, the FINN compiler has a `build_dataflow` tool. Compared to previous versions which required setting up all the needed transformations in a Python script, it makes experimenting with dataflow architecture generation easier. The core idea is to specify the relevant build info as a configuration `dict`, which invokes all the necessary steps to make the dataflow build happen. It can be invoked either from the [command line](https://finn-dev.readthedocs.io/en/latest/command_line.html) or with a single Python function call.


In this notebook, we'll use the Python function call to invoke the builds to stay inside the Jupyter notebook, but feel free to experiment with reproducing what we do here with the `./run-docker.sh build_dataflow` and `./run-docker.sh build_custom` command-line entry points too. 

## Understanding the Build Configuration: `DataflowBuildConfig` <a id="underst_build_conf"></a>

The build configuration is specified by an instance of `finn.builder.build_dataflow_config.DataflowBuildConfig`. The configuration is a Python [`dataclass`](https://docs.python.org/3/library/dataclasses.html) which can be serialized into or de-serialized from JSON files for persistence, although we'll just set it up in Python here.
There are many options in the configuration to customize different aspects of the build, we'll only cover a few of them in this notebook. You can read the details on all the config options on [the FINN API documentation](https://finn-dev.readthedocs.io/en/latest/source_code/finn.builder.html#finn.builder.build_dataflow_config.DataflowBuildConfig).

Let's go over some of the members of the `DataflowBuildConfig`:

### Output Products <a id="output_prod"></a>

The build can produce many different outputs, and some of them can take a long time (e.g. bitfile synthesis for a large network). When you first start working on generating a new accelerator and exploring the different performance options, you may not want to go all the way to a bitfile. Thus, in the beginning you may just select the estimate reports as the output products. Gradually, you can generate the output products from later stages until you are happy enough with the design to build the full accelerator integrated into a shell.

The output products are controlled by:

* `generate_outputs`: list of output products (of type [`finn.builder.build_dataflow_config.DataflowOutputType`](https://finn-dev.readthedocs.io/en/latest/source_code/finn.builder.html#finn.builder.build_dataflow_config.DataflowOutputType)) that will be generated by the build. Some available options are:
    - `ESTIMATE_REPORTS` : report expected resources and performance per layer and for the whole network without any synthesis
    - `STITCHED_IP` : create a stream-in stream-out IP design that can be integrated into other Vivado IPI or RTL designs
    - `RTLSIM_PERFORMANCE` : use PyVerilator to do a performance/latency test of the `STITCHED_IP` design
    - `OOC_SYNTH` : run out-of-context synthesis (just the accelerator itself, without any system surrounding it) on the `STITCHED_IP` design to get post-synthesis FPGA resources and achievable clock frequency
    - `BITFILE` : integrate the accelerator into a shell to produce a standalone bitfile
    - `PYNQ_DRIVER` : generate a PYNQ Python driver that can be used to launch the accelerator
    - `DEPLOYMENT_PACKAGE` : create a folder with the `BITFILE` and `PYNQ_DRIVER` outputs, ready to be copied to the target FPGA platform.
* `output_dir`: the directory where all the generated build outputs above will be written into.
* `steps`: list of predefined (or custom) build steps FINN will go through. Use `build_dataflow_config.estimate_only_dataflow_steps` to execute only the steps needed for estimation (without any synthesis), and the `build_dataflow_config.default_build_dataflow_steps` otherwise (which is the default value). You can find the list of default steps [here](https://finn.readthedocs.io/en/latest/source_code/finn.builder.html#finn.builder.build_dataflow_config.default_build_dataflow_steps) in the documentation.

### Configuring the Board and FPGA Part <a id="config_fpga"></a>

* `fpga_part`: Xilinx FPGA part to be used for synthesis, can be left unspecified to be inferred from `board` below, or specified explicitly for e.g. out-of-context synthesis.
* `board`: target Xilinx Zynq or Alveo board for generating accelerators integrated into a shell. See the `pynq_part_map` and `alveo_part_map` dicts in [this file](https://github.com/Xilinx/finn-base/blob/dev/src/finn/util/basic.py#L41) for a list of possible boards.
* `shell_flow_type`: the target [shell flow type](https://finn-dev.readthedocs.io/en/latest/source_code/finn.builder.html#finn.builder.build_dataflow_config.ShellFlowType), only needed for generating full bitfiles where the FINN design is integrated into a shell (so only needed if `BITFILE` is selected) 

### Configuring the Performance <a id="config_perf"></a>

You can configure the performance (and correspondingly, the FPGA resource footprint) of the generated dataflow accelerator in two ways:

1) (basic) Set a target performance and let the compiler figure out the per-node parallelization settings.

2) (advanced) Specify a separate .json as `folding_config_file` that lists the degree of parallelization (as well as other hardware options) for each layer.

This notebook only deals with the basic approach, for which you need to set up:

* `target_fps`: target inference performance in frames per second. Note that target may not be achievable due to specific layer constraints, or due to resource limitations of the FPGA. 
* `synth_clk_period_ns`: target clock frequency (in nanoseconds) for Vivado synthesis. e.g. `synth_clk_period_ns=5.0` will target a 200 MHz clock. Note that the target clock period may not be achievable depending on the FPGA part and design complexity.

## Launch a Build: Only Estimate Reports <a id="build_estimate_report"></a>

First, we'll launch a build that only generates the estimate reports, which does not require any synthesis. Note two things below: how the `generate_outputs` only contains `ESTIMATE_REPORTS`, but also how the `steps` uses a value of `estimate_only_dataflow_steps`. This skips steps like HLS synthesis to provide a quick estimate from analytical models.

In [21]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_dir = os.environ['FINN_ROOT'] + "/notebooks/end2end_example/cybersecurity"
model_file = model_dir + "/cybsec-mlp-ready.onnx"

estimates_output_dir = "output_estimates_only"

#Delete previous run results if exist
if os.path.exists(estimates_output_dir):
    shutil.rmtree(estimates_output_dir)
    print("Previous run results deleted!")


cfg_estimates = build.DataflowBuildConfig(
    output_dir          = estimates_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    fpga_part           = "xc7a100tcsg324-1",
    steps               = build_cfg.estimate_only_dataflow_steps,    
    rtlsim_batch_size  = 1000,
    default_mem_mode = build_cfg.ComputeEngineMemMode.CONST,
    generate_outputs=[
        build_cfg.DataflowOutputType.ESTIMATE_REPORTS,
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE
    ]
)

Previous run results deleted!


In [28]:
%%time
build.build_dataflow_cfg(model_file, cfg_estimates)

Building dataflow accelerator from /home/afely/Desktop/finn-n/notebooks/end2end_example/cybersecurity/cybsec-mlp-ready.onnx
Intermediate outputs will be generated in /tmp/finn_dev_afely
Final outputs will be generated in output_estimates_only
Build log is at output_estimates_only/build_dataflow.log
Running step: step_qonnx_to_finn [1/9]
Running step: step_tidy_up [2/9]
Running step: step_streamline [3/9]
Running step: step_convert_to_hls [4/9]
Running step: step_create_dataflow_partition [5/9]
Running step: step_target_fps_parallelization [6/9]
Running step: step_apply_folding_config [7/9]
Running step: step_minimize_bit_width [8/9]
Running step: step_generate_estimate_reports [9/9]
Completed successfully
CPU times: user 1.16 s, sys: 136 ms, total: 1.3 s
Wall time: 1.06 s


0

In [29]:
assert os.path.exists(estimates_output_dir + "/report/estimate_network_performance.json")

We'll now examine the generated outputs from this build. If we look under the outputs directory, we'll find a subfolder with the generated estimate reports.

In [30]:
! ls {estimates_output_dir}

auto_folding_config.json  intermediate_models  report  time_per_step.json


In [31]:
! ls {estimates_output_dir}/report

estimate_layer_config_alternatives.json  estimate_network_performance.json
estimate_layer_cycles.json		 op_and_param_counts.json
estimate_layer_resources.json


We see that various reports have been generated as .json files. Let's examine the contents of the `estimate_network_performance.json` for starters. Here, we can see the analytical estimates for the performance and latency.

In [32]:
! cat {estimates_output_dir}/report/estimate_network_performance.json

{
  "critical_path_cycles": 252,
  "max_cycles": 64,
  "max_cycles_node_name": "MatrixVectorActivation_1",
  "estimated_throughput_fps": 1562500.0,
  "estimated_latency_ns": 2520.0
}

Since all of these reports are .json files, we can easily load them into Python for further processing. This can be useful if you are building your own design automation tools on top of FINN. Let's define a helper function and look at the `estimate_layer_cycles.json` report.

In [33]:
import json
def read_json_dict(filename):
    with open(filename, "r") as f:
        ret = json.load(f)
    return ret

In [35]:
read_json_dict(estimates_output_dir + "/report/estimate_layer_cycles.json")

{'MatrixVectorActivation_0': 60,
 'MatrixVectorActivation_1': 64,
 'MatrixVectorActivation_2': 64,
 'MatrixVectorActivation_3': 64}

Here, we can see the estimated number of clock cycles each layer will take. Recall that all of these layers will be running in parallel, and the slowest layer will determine the overall throughput of the entire neural network. FINN attempts to parallelize each layer such that they all take a similar number of cycles, and less than the corresponding number of cycles that would be required to meet `target_fps`. Additionally by summing up all layer cycle estimates one can obtain an estimate for the overall latency of the whole network. 

Finally, we can see the layer-by-layer resource estimates in the `estimate_layer_resources.json` report:

In [36]:
read_json_dict(estimates_output_dir + "/report/estimate_layer_resources.json")

{'MatrixVectorActivation_0': {'BRAM_18K': 0,
  'BRAM_efficiency': 1,
  'LUT': 8021,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'MatrixVectorActivation_1': {'BRAM_18K': 0,
  'BRAM_efficiency': 1,
  'LUT': 1277,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'MatrixVectorActivation_2': {'BRAM_18K': 0,
  'BRAM_efficiency': 1,
  'LUT': 1277,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'MatrixVectorActivation_3': {'BRAM_18K': 0,
  'BRAM_efficiency': 1,
  'LUT': 319,
  'URAM': 0,
  'URAM_efficiency': 1,
  'DSP': 0},
 'total': {'BRAM_18K': 0.0, 'LUT': 10894.0, 'URAM': 0.0, 'DSP': 0.0}}

This particular report is useful to determine whether the current configuration will fit into a particular FPGA. If you see that the resource requirements are too high for the FPGA you had in mind, you should consider lowering the `target_fps`.

**Note that the analytical models tend to over-estimate how much resources are needed, since they can't capture the effects of various synthesis optimizations.**

## Launch a Build: Stitched IP, out-of-context synth and rtlsim Performance <a id="build_ip_synth_rtlsim"></a>

Once we have a configuration that gives satisfactory estimates, we can move on to generating the accelerator. We can do this in different ways depending on how we want to integrate the accelerator into a larger system. For instance, if we have a larger streaming system built in Vivado or if we'd like to re-use this generated accelerator as an IP component in other projects, the `STITCHED_IP` output product is a good choice. We can also use the `OOC_SYNTH` output product to get post-synthesis resource and clock frequency numbers for our accelerator.

<font color="red">**Live FINN tutorial:** These next builds will take about 10 minutes to complete since multiple calls to Vivado and a call to RTL simulation are involved. While this is running, you can examine the generated files with noVNC -- it is running on **(your AWS URL):6080/vnc.html**

* Once the `step_hls_codegen [8/16]` below is completed, you can view the generated HLS code under its own folder for each layer: `/tmp/finn_dev_ubuntu/code_gen_ipgen_MatrixVectorActivation_XXXXXX`
    
* Once the `step_create_stitched_ip [11/16]` below is completed, you can view the generated stitched IP in Vivado under `/home/ubuntu/finn/notebooks/end2end_example/cybersecurity/output_ipstitch_ooc_rtlsim/stitched_ip`
</font> 

In [37]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = model_dir + "/cybsec-mlp-ready.onnx"

rtlsim_output_dir = "output_ipsmodel_filetitch_ooc_rtlsim"

#Delete previous run results if exist
if os.path.exists(rtlsim_output_dir):
    shutil.rmtree(rtlsim_output_dir)
    print("Previous run results deleted!")
rtlsim_output_dir
cfg_stitched_ip = build.DataflowBuildConfig(
    output_dir          = rtlsim_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    fpga_part           = "xc7a100tcsg324-1",
    rtlsim_batch_size  = 1000,
    default_mem_mode = build_cfg.ComputeEngineMemMode.CONST,
    generate_outputs=[
        build_cfg.DataflowOutputType.STITCHED_IP,
        build_cfg.DataflowOutputType.RTLSIM_PERFORMANCE,
        build_cfg.DataflowOutputType.OOC_SYNTH,
    ]
)

In [38]:
%%time
build.build_dataflow_cfg(model_file, cfg_stitched_ip)

Building dataflow accelerator from /home/afely/Desktop/finn-n/notebooks/end2end_example/cybersecurity/cybsec-mlp-ready.onnx
Intermediate outputs will be generated in /tmp/finn_dev_afely
Final outputs will be generated in output_ipsmodel_filetitch_ooc_rtlsim
Build log is at output_ipsmodel_filetitch_ooc_rtlsim/build_dataflow.log
Running step: step_qonnx_to_finn [1/18]
Running step: step_tidy_up [2/18]
Running step: step_streamline [3/18]
Running step: step_convert_to_hls [4/18]
Running step: step_create_dataflow_partition [5/18]
Running step: step_target_fps_parallelization [6/18]
Running step: step_apply_folding_config [7/18]
Running step: step_minimize_bit_width [8/18]
Running step: step_generate_estimate_reports [9/18]
Running step: step_hls_codegen [10/18]
Running step: step_hls_ipgen [11/18]
Running step: step_set_fifo_depths [12/18]
Running step: step_create_stitched_ip [13/18]
Running step: step_measure_rtlsim_performance [14/18]
Running step: step_out_of_context_synthesis [15/18

0

In [39]:
assert os.path.exists(rtlsim_output_dir + "/report/ooc_synth_and_timing.json")
assert os.path.exists(rtlsim_output_dir + "/report/rtlsim_performance.json")
assert os.path.exists(rtlsim_output_dir + "/final_hw_config.json")

Why is e.g. `step_synthesize_bitfile` listed above even though we didn't ask for a bitfile in the output products? This is because we're using the default set of build steps, which includes `step_synthesize_bitfile`. Since its output product is not selected, this step will do nothing.

Among the output products, we will find the accelerator exported as a stitched IP block design:

In [40]:
! ls {rtlsim_output_dir}/stitched_ip

all_verilog_srcs.txt		       finn_vivado_stitch_proj.xpr
data				       ip
finn_vivado_stitch_proj.cache	       make_project.sh
finn_vivado_stitch_proj.gen	       make_project.tcl
finn_vivado_stitch_proj.hw	       vivado.jou
finn_vivado_stitch_proj.ip_user_files  vivado.log
finn_vivado_stitch_proj.srcs


We also have a few reports generated by these output products, different from the ones generated by `ESTIMATE_REPORTS`.

In [41]:
! ls {rtlsim_output_dir}/report

estimate_layer_resources_hls.json  rtlsim_performance.json
ooc_synth_and_timing.json


In `ooc_synth_and_timing.json` we can find the post-synthesis and maximum clock frequency estimate for the accelerator. Note that the clock frequency estimate here tends to be optimistic, since out-of-context synthesis is less constrained.

In [42]:
! cat {rtlsim_output_dir}/report/ooc_synth_and_timing.json

{
  "vivado_proj_folder": "/tmp/finn_dev_afely/synth_out_of_context_3th0r5c6/results_finn_design_wrapper",
  "LUT": 5960.0,
  "LUTRAM": 44.0,
  "FF": 5231.0,
  "DSP": 0.0,
  "BRAM": 0.0,
  "BRAM_18K": 0.0,
  "BRAM_36K": 0.0,
  "URAM": 0.0,
  "Carry": 303.0,
  "WNS": 0.773,
  "Delay": 0.773,
  "vivado_version": 2023.2,
  "vivado_build_no": 4029153.0,
  "": 0,
  "fmax_mhz": 108.37758751490192,
  "estimated_throughput_fps": 1693399.8049203425
}

In `rtlsim_performance.json` we can find the steady-state throughput and latency for the accelerator, as obtained by rtlsim. If the DRAM bandwidth numbers reported here are below what the hardware platform is capable of (i.e. the accelerator is not memory-bound), you can expect the same steady-state throughput (excluding any software/driver overheads) in real hardware.

In [43]:
! cat {rtlsim_output_dir}/report/rtlsim_performance.json

{
  "N_IN_TXNS": 15000,
  "N_OUT_TXNS": 1000,
  "cycles": 73140,
  "N": 1000,
  "latency_cycles": 213,
  "runtime[ms]": 0.7314,
  "throughput[images/s]": 1367240.9078479628,
  "fclk[mhz]": 100.0,
  "stable_throughput[images/s]": 1371234.2479465767
}

Finally, let's have a look at `final_hw_config.json`. This is the node-by-node hardware configuration determined by the FINN compiler, including FIFO depths, parallelization settings (PE/SIMD) and others. If you want to optimize your build further (the "advanced" method we mentioned under "Configuring the performance"), you can use this .json file as the `folding_config_file` for a new run to use it as a starting point for further exploration and optimizations.

In [46]:
! cat {rtlsim_output_dir}/final_hw_config.json

{
  "Defaults": {},
  "StreamingFIFO_0": {
    "ram_style": "auto",
    "depth": 32,
    "impl_style": "rtl",
    "inFIFODepths": [
      0
    ],
    "outFIFODepths": [
      0
    ]
  },
  "MatrixVectorActivation_0": {
    "PE": 16,
    "SIMD": 40,
    "ram_style": "auto",
    "resType": "lut",
    "mem_mode": "const",
    "runtime_writeable_weights": 0,
    "inFIFODepths": [
      32
    ],
    "outFIFODepths": [
      0
    ]
  },
  "StreamingDataWidthConverter_rtl_0": {
    "inFIFODepths": [
      0
    ],
    "outFIFODepths": [
      0
    ]
  },
  "MatrixVectorActivation_1": {
    "PE": 1,
    "SIMD": 64,
    "ram_style": "auto",
    "resType": "lut",
    "mem_mode": "const",
    "runtime_writeable_weights": 0,
    "inFIFODepths": [
      0
    ],
    "outFIFODepths": [
      0
    ]
  },
  "StreamingDataWidthConverter_rtl_1": {
    "inFIFODepths": [
      0
    ],
    "outFIFODepths": [
      0
    ]
  },
  "MatrixVectorActivation_2": {
    "PE": 1,
    "SIMD": 64,
    "ram_sty

In [64]:
! ls {rtlsim_output_dir}/intermediate_models/

dataflow_parent.onnx		     step_measure_rtlsim_performance.onnx
step_apply_folding_config.onnx	     step_minimize_bit_width.onnx
step_convert_to_hls.onnx	     step_out_of_context_synthesis.onnx
step_create_dataflow_partition.onnx  step_qonnx_to_finn.onnx
step_create_stitched_ip.onnx	     step_set_fifo_depths.onnx
step_deployment_package.onnx	     step_streamline.onnx
step_generate_estimate_reports.onnx  step_synthesize_bitfile.onnx
step_hls_codegen.onnx		     step_target_fps_parallelization.onnx
step_hls_ipgen.onnx		     step_tidy_up.onnx
step_make_pynq_driver.onnx	     supported_op_partitions


In [71]:
from finn.core.onnx_exec import execute_onnx
from qonnx.core.modelwrapper import ModelWrapper
from finn.util.visualization import showInNetron
import torch 

rtlsim_model = ModelWrapper(rtlsim_output_dir + "/intermediate_models/step_create_stitched_ip.onnx")
showInNetron(rtlsim_output_dir + "/intermediate_models/step_create_stitched_ip.onnx")
rtlsim_model.set_metadata_prop("exec_mode", "rtlsim")


Stopping http://0.0.0.0:8081
Serving 'output_ipsmodel_filetitch_ooc_rtlsim/intermediate_models/step_create_stitched_ip.onnx' at http://0.0.0.0:8081


In [101]:
def test_rtlsim_model_on_dataset(model, test_loader):
    y_true = []
    y_pred = []

    for data in test_loader:
        inputs, target = data
        batch_size = inputs.shape[0]
        model.set_tensor_shape("global_in", (batch_size, 600))
        model.set_tensor_shape("global_out", (batch_size, 1))
        output = execute_onnx(model, {"global_in": inputs.float().numpy()})["global_out"]
        y_pred.extend(list(output.flatten()))
        y_true(list(target.detach().numpy().flatten()))

    return accuracy_score(y_true, y_pred)

In [102]:
import numpy as np
from torch.utils.data import TensorDataset

def get_preqnt_dataset(data_dir: str, train: bool):
    unsw_nb15_data = np.load(data_dir + "/unsw_nb15_binarized.npz")
    if train:
        partition = "train"
    else:
        partition = "test"
    part_data = unsw_nb15_data[partition].astype(np.float32)
    print("Carregando como um vetor unico de " + str(len(part_data)) + " linhas e " + str(len(part_data[0])) + " colunas (bit wide)")
    print(part_data)
    part_data = torch.from_numpy(part_data)
    print("\nConvertendo para um tensor com mesma dimensão...")
    print(part_data)
    #os códigos a seguir servem para separar a entrada(vetor de 593) e a sua label
    part_data_in = part_data[:, :-1] #vetor de 593 bits
    part_data_out = part_data[:, -1] #label
    print("-------------------------------------------------------")
    return TensorDataset(part_data_in, part_data_out)

train_quantized_dataset = get_preqnt_dataset(".", True)
test_quantized_dataset = get_preqnt_dataset(".", False)

print("Samples in each set: train = %d, test = %s" % (len(train_quantized_dataset), len(test_quantized_dataset))) 
print("Shape of one input sample: " +  str(train_quantized_dataset[0][0].shape))

Carregando como um vetor unico de 175341 linhas e 594 colunas (bit wide)
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]

Convertendo para um tensor com mesma dimensão...
tensor([[0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 0., 0., 1.]])
-------------------------------------------------------
Carregando como um vetor unico de 82332 linhas e 594 colunas (bit wide)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]]

Convertendo para um tensor com mesma dimensão...
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        ...,


In [103]:
from torch.utils.data import DataLoader, Dataset

#batch: é o tamanho do lote de entrada que eu to fornecendo pra minha rede neural processar
batch_size = 1000

# dataset loaders
train_quantized_loader = DataLoader(train_quantized_dataset, batch_size=batch_size, shuffle=True)
test_quantized_loader = DataLoader(test_quantized_dataset, batch_size=batch_size, shuffle=False)  

In [104]:
test_rtlsim_model_on_dataset(rtlsim_model, test_quantized_loader)

Exception: Shape mismatch for provided input global_in: found (1000, 600) expected (1000, 593) 